In [ ]:
# Target Encoding
# 我们在本课程中看到的大多数技术都是针对数值特征的。我们将在本课中介绍的技术（目标编码）是针对分类特征的。
# 它是一种将类别编码为数字的方法，如单热编码或标签编码，不同之处在于它还使用目标来创建编码。
# 这使它成为我们所说的监督特征工程技术。

In [ ]:
import pandas as pd

autos = pd.read_csv("../input/fe-course-data/autos.csv")

In [ ]:
# 目标编码¶
# 目标编码是将要素的类别替换为从目标派生的某个数字的任何类型的编码。

# 一个简单而有效的版本是应用第 3 课中的组聚合，例如平均值。使用汽车数据集，可以计算每辆车的平均价格：
autos["make_encoded"] = autos.groupby("make")["price"].transform("mean")

autos[["make", "price", "make_encoded"]].head(10)

# 这种目标编码有时称为均值编码。应用于二进制目标，也称为箱数计数。（您可能会遇到的其他名称包括：似然编码、影响编码和遗漏一例外编码。

In [ ]:
# 平滑¶
# 组转换形成的编码会带来一些问题。首先是未知类别。
# 目标编码会产生过拟合的特殊风险，这意味着它们需要在独立的“编码”拆分上进行训练。
# 当您将编码加入到将来的拆分中时，Pandas 将填充编码拆分中不存在的任何类别的缺失值。这些缺失的值，你必须以某种方式插补。

# 其次是稀有类别。当一个类别在数据集中只出现几次时，根据其组计算的任何统计数据都不太可能非常准确。
# 在 Automobiles 数据集中，mercurcy make 仅发生一次。
# 我们计算的“平均”价格只是这辆车的价格，它可能不太能代表我们将来可能看到的任何 Mercuries。对稀有类别进行目标编码会使过度拟合的可能性更大。

# 解决这些问题的方法是添加平滑。这个想法是将类别内平均值与总体平均值混合在一起。
# 稀有类别在其类别平均值上的权重较小，而缺失类别仅获得总体平均值。

# 在伪代码中：
'encoding = weight * in_category + (1 - weight) * overall'
# 'in_category': 类别内平均值(在这个示例中)
# 其中，权重是根据类别频率计算得出的介于 0 和 1 之间的值。

# 确定权重值的一种简单方法是计算 m 估计值：
'weight = n / (n + m)'

# 其中 n 是该类别在数据中出现的总次数。参数 m 确定“平滑因子”。m 值越大，总体估计值的权重越大。

In [ ]:
# 目标编码的用例
# 目标编码非常适合：
# 高基数特征：具有大量类别的特征可能很难编码：单热编码会生成太多特征，而替代方法（如标签编码）可能不适合该特征。
# 目标编码使用要素最重要的属性（其与目标的关系）派生类别的编号。
# 领域激励功能：根据以前的经验，您可能会怀疑分类功能应该很重要，即使它在功能指标中得分很低。
# 目标编码可以帮助揭示特征的真实信息性。

In [ ]:
# MovieLens1M
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
warnings.filterwarnings('ignore')


df = pd.read_csv("../input/fe-course-data/movielens1m.csv")
df = df.astype(np.uint8, errors='ignore') # reduce memory footprint
print("Number of Unique Zipcodes: {}".format(df["Zipcode"].nunique()))

In [ ]:
# With over 3000 categories, the Zipcode feature makes a good candidate for target encoding, 
# and the size of this dataset (over one-million rows) means we can spare some data to create the encoding.

# We'll start by creating a 25% split to train the target encoder.

X = df.copy()
y = X.pop('Rating')

X_encode = X.sample(frac=0.25)
y_encode = y[X_encode.index]
X_pretrain = X.drop(X_encode.index)
y_train = y[X_pretrain.index]

In [ ]:
# The category_encoders package in scikit-learn-contrib implements an m-estimate encoder, 
# which we'll use to encode our Zipcode feature.
from category_encoders import MEstimateEncoder

# Create the encoder instance. Choose m to control noise.
encoder = MEstimateEncoder(cols=["Zipcode"], m=5.0)

# Fit the encoder on the encoding split.
encoder.fit(X_encode, y_encode)

# Encode the Zipcode column to create the final training data
X_train = encoder.transform(X_pretrain)

In [ ]:
# Let's compare the encoded values to the target to see how informative our encoding might be.
plt.figure(dpi=90)
ax = sns.distplot(y, kde=False, norm_hist=True)
ax = sns.kdeplot(X_train.Zipcode, color='r', ax=ax)
ax.set_xlabel("Rating")
ax.legend(labels=['Zipcode', 'Rating']);

# 编码的邮政编码功能的分布大致遵循实际评级的分布，
# 这意味着电影观众的评级在邮政编码之间的差异很大，以至于我们的目标编码能够捕获有用的信息。

In [ ]:
# ex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from category_encoders import MEstimateEncoder
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
warnings.filterwarnings('ignore')


def score_dataset(X, y, model=XGBRegressor()):
    # Label encoding for categoricals
    for colname in X.select_dtypes(["category", "object"]):
        X[colname], _ = X[colname].factorize()
    # Metric for Housing competition is RMSLE (Root Mean Squared Log Error)
    score = cross_val_score(
        model, X, y, cv=5, scoring="neg_mean_squared_log_error",
    )
    score = -1 * score.mean()
    score = np.sqrt(score)
    return score


df = pd.read_csv("../input/fe-course-data/ames.csv")

In [ ]:
# First you'll need to choose which features you want to apply a target encoding to. 
# Categorical features with a large number of categories are often good candidates. 
# Run this cell to see how many categories each categorical feature in the Ames dataset has.
df.select_dtypes(["object"]).nunique()

In [ ]:
# We talked about how the M-estimate encoding uses smoothing to improve estimates for rare categories. 
# To see how many times a category occurs in the dataset, you can use the value_counts method. 
# This cell shows the counts for SaleType, but you might want to consider others as well.
df["SaleType"].value_counts()

In [ ]:
# Choose Features for Encoding
# Which features did you identify for target encoding? 

# 每次训练都要单独创造训练集来训练
# Now you'll apply a target encoding to your choice of feature. 
# As we discussed in the tutorial, to avoid overfitting, we need to fit the encoder on data heldout from the training set. 
# Run this cell to create the encoding and training splits:


# Encoding split
X_encode = df.sample(frac=0.20, random_state=0)
y_encode = X_encode.pop("SalePrice")

# Training split
X_pretrain = df.drop(X_encode.index)
y_train = X_pretrain.pop("SalePrice")

In [ ]:
# Apply M-Estimate Encoding
# Apply a target encoding to your choice of categorical features. 
# Also choose a value for the smoothing parameter m (any value is okay for a correct answer).
# Choose a set of features to encode and a value for m
encoder = MEstimateEncoder(cols=["Neighborhood"], m=3.0)


# Fit the encoder on the encoding split
encoder.fit(X_encode, y_encode)

# Encode the training split
X_train = encoder.transform(X_pretrain, y_train)

In [ ]:

feature = encoder.cols

plt.figure(dpi=90)
ax = sns.distplot(y_train, kde=True, hist=False)
ax = sns.distplot(X_train[feature], color='r', ax=ax, hist=True, kde=False, norm_hist=True)
ax.set_xlabel("SalePrice");

In [ ]:
# compare
X = df.copy()
y = X.pop("SalePrice")
score_base = score_dataset(X, y)
score_new = score_dataset(X_train, y_train)

print(f"Baseline Score: {score_base:.4f} RMSLE")
print(f"Score with Encoding: {score_new:.4f} RMSLE")

In [ ]:
# Do you think that target encoding was worthwhile in this case? 
# Depending on which feature or features you chose, you may have ended up with a score significantly worse than the baseline. 
# In that case, it's likely the extra information gained by the encoding couldn't make up for the loss of data used for the encoding.

# In this question, you'll explore the problem of overfitting with target encodings. This will illustrate this importance of training fitting target encoders on data held-out from the training set.

# So let's see what happens when we fit the encoder and the model on the same dataset. 
# To emphasize how dramatic the overfitting can be, we'll mean-encode a feature that should have no relationship with SalePrice, a count: 0, 1, 2, 3, 4, 5, ....

# Try experimenting with the smoothing parameter m
# Try 0, 1, 5, 50
m = 50

X = df.copy()
y = X.pop('SalePrice')

# Create an uninformative feature
X["Count"] = range(len(X))
X["Count"][1] = 0  # actually need one duplicate value to circumvent error-checking in MEstimateEncoder

# fit and transform on the same dataset
encoder = MEstimateEncoder(cols="Count", m=m)
X = encoder.fit_transform(X, y)

# Results
score =  score_dataset(X, y)
print(f"Score: {score:.4f} RMSLE")

In [ ]:
plt.figure(dpi=90)
ax = sns.distplot(y, kde=True, hist=False)
ax = sns.distplot(X["Count"], color='r', ax=ax, hist=True, kde=False, norm_hist=True)
ax.set_xlabel("SalePrice");

# Since Count never has any duplicate values, the mean-encoded Count is essentially an exact copy of the target. In other words, mean-encoding turned a completely meaningless feature into a perfect feature.

# Now, the only reason this worked is because we trained XGBoost on the same set we used to train the encoder. If we had used a hold-out set instead, none of this "fake" encoding would have transferred to the training data.

# The lesson is that when using a target encoder it's very important to use separate data sets for training the encoder and training the model. Otherwise the results can be very disappointing!